# LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, `Runnable` 프로토콜을 구현했습니다.

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다. 표준 인터페이스에는 다음이 포함됩니다.

- `stream`: 응답의 청크를 스트리밍합니다.
- `invoke`: 입력에 대해 체인을 호출합니다.
- `batch`: 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- `astream`: 비동기적으로 응답의 청크를 스트리밍합니다.
- `ainvoke`: 비동기적으로 입력에 대해 체인을 호출합니다.
- `abatch`: 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- `astream_log`: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI(model="gpt-4o-mini")
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

# stream: 실시간 출력

이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터를 스트리밍 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다[1]. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다[1].



In [4]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 유형의 데이터를 통합하여 처리하는 방식으로, 예를 들어 텍스트, 이미지, 음성 등의 다양한 정보를 함께 사용합니다. 이러한 접근은 정보의 풍부성을 높이고, 더 나은 이해와 분석을 가능하게 합니다. 최근에는 인공지능과 머신러닝 분야에서 멀티모달 학습이 활발히 연구되어, 보다 정교한 모델 개발에 기여하고 있습니다.

In [5]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자가 입력한 질문이나 요청에 대해 유연하게 응답하며, 다양한 주제에 대한 정보를 제공할 수 있습니다. 이 모델은 대화의 맥락을 이해하고 적절한 답변을 생성하는 데 강력한 성능을 발휘합니다.'

In [6]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 OpenAI에서 개발한 자연어 처리 AI 모델로, 인간처럼 대화할 수 있는 능력을 가지고 있습니다. 사용자가 입력하는 질문이나 요청에 대해 문맥에 맞는 응답을 생성하여 대화의 흐름을 이어갑니다. 다양한 주제에 대한 정보 제공, 문제 해결, 창의적인 글쓰기 등 다양한 용도로 활용될 수 있습니다.',
 '인스타그램은 사용자가 사진과 동영상을 공유하고 소통할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 통해 사용자들은 자신의 콘텐츠를 창의적으로 표현할 수 있으며, 해시태그를 통해 다양한 주제의 콘텐츠를 탐색할 수 있습니다. 또한, 친구와의 소통은 물론, 유명 인플루언서와 브랜드와의 연결을 통해 새로운 트렌드를 발견하는 데 도움을 줍니다.']

max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [7]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 OpenAI가 개발한 인공지능 언어 모델로, 자연어 처리 기술을 기반으로 합니다. 이 모델은 다양한 주제에 대해 대화를 생성하고 질문에 답변할 수 있도록 설계되었습니다. 사용자는 ChatGPT를 통해 정보를 얻거나 창의적인 작업을 수행하는 등 여러 방식으로 활용할 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 창의적인 콘텐츠 제작을 지원하며, 친구들과의 소통뿐만 아니라 넷플릭스, 스타벅스 등 유명 브랜드와의 상호작용도 활발합니다. 또한, 스토리, IGTV, 리일즈 등의 기능을 통해 다양한 형식의 콘텐츠를 제작하고 소비할 수 있는 환경을 제공합니다.',
 '멀티모달(Multimodal)은 다양한 유형의 데이터나 정보(예: 텍스트, 이미지, 음성 등)를 결합하여 분석하거나 처리하는 접근 방식을 의미합니다. 이 방식은 인간의 인지 방식과 유사하게 여러 감각을 통합하여 더 풍부한 정보를 제공할 수 있습니다. 최근 인공지능 분야에서는 멀티모달 학습이 중요해지며, 서로 다른 모달리티 간의 상호작용을 통해 더 높은 성능을 이끌어내고 있습니다.',
 '프로그래밍은 컴퓨터에게 특정 작업을 수행하도록 명령하는 과정으로, 주로 프로그래밍 언어를 사용하여 코드를 작성합니다. 이 과정에서는 문제를 분석하고, 알고리즘을 설계하며, 코드를 구현하는 단계가 포함됩니다. 프로그래밍 기술은 소프트웨어 개발, 웹 디자인, 데이터 분석 등 다양한 분야에서 필수적입니다.',
 '머신러닝은 데이터에서 패턴을 학습하여 예측이나 결정을 내리는 기술입니다. 알고리즘은 주어진 입력 데이터를 분석하고, 이를 통해 모델을 개선하며 성능을 향상시킵니다. 다양한 분야에서 활용되며, 이미지 인식, 자연어 처리, 추천 시스템 등 여러 응용 프로그램에 적용됩니다.']

async stream: 비동기 스트림
함수 chain.astream은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(async for)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, print 함수를 통해 메시지의 내용(s.content)을 즉시 출력합니다. end=""는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.

In [8]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 사용자가 비디오를 업로드, 공유 및 시청할 수 있는 온라인 플랫폼입니다. 2005년에 설립된 이 서비스는 다양한 콘텐츠 creators들이 음악, 교육, 엔터테인먼트 등 다양한 주제의 비디오를 제작하고 방송할 수 있는 공간을 제공합니다. 또한, 사용자들은 구독, 댓글, 좋아요와 같은 기능을 통해 서로 소통하며, 개인 맞춤형 추천 알고리즘을 통해 관심 있는 콘텐츠를 쉽게 찾을 수 있습니다.

async invoke: 비동기 호출
chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 topic이라는 키와 NVDA(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.

In [9]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [10]:
await my_process

'NVIDIA Corporation(NVDA)는 그래픽 처리 장치(GPU)와 인공지능(AI) 기술을 주로 개발하는 미국의 기술 기업입니다. 이 회사는 게임, 데이터 센터, 인공지능 및 자율주행차 등 다양한 분야에 강력한 솔루션을 제공하고 있으며, 최근에는 AI 및 머신러닝 시장에서 큰 성장을 이루고 있습니다. NVIDIA의 혁신적인 제품군은 고성능 컴퓨팅 및 비주얼 컴퓨팅 분야에서 널리 사용되고 있습니다.'

async batch: 비동기 배치
함수 abatch는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 chain 객체의 abatch 메서드를 사용하여 topic 에 대한 작업을 비동기적으로 처리하고 있습니다.

await 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.

In [11]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [12]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 사용자들이 비디오 콘텐츠를 업로드, 시청, 공유할 수 있는 온라인 플랫폼입니다. 다양한 주제의 콘텐츠가 있으며, 개인 사용자뿐만 아니라 기업과 기관도 활용하여 마케팅 및 정보 전달을 합니다. 또한, 사용자들은 구독, 좋아요, 댓글 등으로 상호작용하며 커뮤니티를 형성할 수 있습니다.',
 '인스타그램은 사용자들이 사진과 동영상을 공유하고 소통할 수 있는 소셜 미디어 플랫폼입니다. 사용자는 다양한 필터와 편집 도구를 이용해 콘텐츠를 꾸미고, 해시태그를 통해 더 많은 사람들과 연결될 수 있습니다. 또한, 스토리, IGTV, 릴스와 같은 기능을 통해 실시간으로 일상을 공유하거나 창의적인 콘텐츠를 제작할 수 있습니다.',
 'Facebook은 2004년 마크 저커버그와 그의 친구들이 대학생들을 위해 만든 소셜 미디어 플랫폼으로, 사람들이 서로 소통하고 정보를 공유할 수 있는 공간입니다. 사용자는 프로필을 만들어 친구를 추가하고, 게시물을 작성하며 사진과 비디오를 공유할 수 있습니다. 현재 Facebook은 전 세계적으로 수억 명의 사용자를 보유하고 있으며, 다양한 비즈니스와 광고 기회를 제공하는 중요한 플랫폼으로 자리잡고 있습니다.']

Parallel: 병렬성
LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다. 예를 들어, RunnableParallel을 사용할 때(자주 사전 형태로 작성됨), 각 요소를 병렬로 실행합니다.

langchain_core.runnables 모듈의 RunnableParallel 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

ChatPromptTemplate.from_template 메서드를 사용하여 주어진 country에 대한 수도 와 면적 을 구하는 두 개의 체인(chain1, chain2)을 만듭니다.

이 체인들은 각각 model과 파이프(|) 연산자를 통해 연결됩니다. 마지막으로, RunnableParallel 클래스를 사용하여 이 두 체인을 capital와 area이라는 키로 결합하여 동시에 실행할 수 있는 combined 객체를 생성합니다.

In [13]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

In [14]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울입니다.'

In [15]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,830,000 평방킬로미터(3,796,742 평방마일)입니다. 이는 미국이 세계에서 세 번째로 큰 나라임을 의미합니다.'

In [16]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울입니다.',
 'area': '대한민국의 면적은 약 100,210 평방 킬로미터입니다. 이는 한국 전체의 면적을 포함한 값으로, 한반도의 남쪽 부분에 해당합니다.'}

배치에서의 병렬 처리
병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다. 배치와 병렬 처리를 사용해 보도록 합시다.

chain1.batch 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.

In [17]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

In [18]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 면적은 약 100,210 평방킬로미터입니다. 이는 한반도의 남쪽에 위치한 지역을 포함한 면적입니다.',
 '미국의 면적은 약 9,830,000 평방킬로미터(3,796,000 평방마일)입니다. 이는 미국이 세계에서 세 번째로 큰 국가라는 것을 의미합니다.']

In [19]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울입니다.',
  'area': '대한민국의 면적은 약 100,210 평방킬로미터(약 38,691 평방 마일)입니다. 이 면적에는 한반도의 남쪽 부분과 다양한 섬들이 포함되어 있습니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '미국의 면적은 약 9,826,675 평방킬로미터(약 3,796,742 평방 마일)입니다. 이 면적에는 주 및 영토가 포함되어 있습니다. 미국은 세계에서 세 번째로 큰 국가로, 넓은 다양성을 가진 지형과 생태계를 보유하고 있습니다.'}]